PDF2docx Converter

In [288]:
from pdf2docx import Converter

In [498]:
pdf_file = 'Stanford.pdf' # enter name of the pdf here
word_file = 'test-1.docx'

cv = Converter(pdf_file)
cv.convert(word_file, start = 20,end = 80)

cv.close() # run this code to convert pdf

[INFO] Start to convert Stanford.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/60) Page 21
[INFO] (2/60) Page 22
[INFO] (3/60) Page 23
[INFO] (4/60) Page 24
[INFO] (5/60) Page 25
[INFO] (6/60) Page 26
[INFO] (7/60) Page 27
[INFO] (8/60) Page 28
[INFO] (9/60) Page 29
[INFO] (10/60) Page 30
[INFO] (11/60) Page 31
[INFO] (12/60) Page 32
[INFO] (13/60) Page 33
[INFO] (14/60) Page 34
[INFO] (15/60) Page 35
[INFO] (16/60) Page 36
[INFO] (17/60) Page 37
[INFO] (18/60) Page 38
[INFO] (19/60) Page 39
[INFO] (20/60) Page 40
[INFO] (21/60) Page 41
[INFO] (22/60) Page 42
[INFO] (23/60) Page 43
[INFO] (24/60) Page 44
[INFO] (25/60) Page 45
[INFO] (26/60) Page 46
[INFO] (27/60) Page 47
[INFO] (28/60) Page 48
[INFO] (29/60) Page 49
[INFO] (30/60) Page 50
[INFO] (31/60) Page 51
[INFO] (32/60) Page 52
[INFO] (33/60) Page 53
[INFO] (34/60) Page 54
[INFO] (35/60) Page 55
[INFO] (36/60) Page 56
[INFO] (37/60) Page 57
[INFO] (38/60) Page 58


In [499]:
from docx import Document # run this code next
import pandas as pd
document = Document('test-1.docx')

tables = []
for table in document.tables:
    df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
    for i, row in enumerate(table.rows):
        for j, cell in enumerate(row.cells):
            if cell.text:
                df[i][j] = cell.text
    tables.append(pd.DataFrame(df))

Run all of the functions below

In [312]:
def find_metric(df, metric):
    for i in range(len(df.index)):
        if df.iloc[i,].str.contains(metric, na = False, case = False).any():
            return(df)

In [313]:
def check_table_contain_metric(table, metric):
    i = 0
    while i < len(table.index):
        if table.iloc[i,].str.contains(metric, na = False, case = False).any() == True:
            return True
            break
        else:
            i += 1
    return False

In [314]:
def return_tables_contain_metric(tables, metric):
    a = []
    for t in tables:
        if check_table_contain_metric(t,metric) == True:
            a.append(find_metric(t,metric))
        
    return a

In [436]:
def table_orientation(df):
    if len(df.index) <= 1:
        return('Does not contain number')
    if len(df.columns) <= 1:
        return('Does not contain number')
    if len(df.index) > 1:
        value_a = df.iloc[0,len(df.columns)-1]
        value_b = df.iloc[len(df.index)-1, 0]
        
        try: 
            float(value_a.replace(',','').replace('(','-').replace(')','').replace(' ','').replace('*','').replace('$','').replace('%','').replace('\n','').replace('-','').replace('–',''))
            return('Vertical')
        except ValueError:
                try:
                    float(value_b.replace(',','').replace('(','-').replace(')','').replace(' ','').replace('*','').replace('$','').replace('%','').replace('\n','').replace('-','').replace('–',''))
                    return ('Horizontal')
                except ValueError:
                    return('Does not contain number')             

In [437]:
def get_value(df, metric):
    if table_orientation(df) == 'Horizontal':
        for i in range(len(df.columns)):
                if df.iloc[:,i].str.contains(metric, na = False, case = False).any():
                    value =  df.iloc[len(df.index)-1, i]
                    try: 
                        num = float(value.replace(',','').replace('(','-').replace(')','').replace(' ','').replace('*','').replace('$','').replace('%','').replace('\n','').replace('-','').replace('–',''))
                        return(num)
                    except ValueError:
                        try:
                            value =  df.iloc[len(df.index)-2, i]
                            num = float(value.replace(',','').replace('(','-').replace(')','').replace(' ','').replace('*','').replace('$','').replace('%','').replace('\n','').replace('-','').replace('–',''))
                            return(num)
                        except ValueError:
                            return('N/A cannot extract number due to format')
    
    elif table_orientation(df) == 'Vertical':
        for j in range(len(df.index)):
            if df.iloc[j,:].str.contains(metric, na = False, case = False).any():
                value = df.iloc[j,len(df.columns)-1]
                try:
                    num = float(value.replace(',','').replace('(','-').replace(')','').replace(' ','').replace('*','').replace('$','').replace('%','').replace('\n','').replace('-','').replace('–',''))
                    return(num)

                except ValueError:
                    return('N/A cannot extract number due to format')

    else:
        return('N/A')


In [492]:
def get_metric(tables, metric):
    shortlist = return_tables_contain_metric(tables,metric)
    a=[]
    for t in shortlist:
        if get_value(t, metric) != 'N/A':
            a.append(get_value(t,metric))
    if len(a) <1:
        return 'N/A'
    else:    
        return(a[0])

In [1]:
metrics = ['Application','Enroll','Accepted','SAT','Tuition','Total operating revenues','Total operating expenses','Net Assets', 'Net income', 'Net operating surplus',
 'Cash','Depreciation', 'Interest','Investment']

 # List of metrics to be extracted. These are considered "search terms" - we can modify the terms to best match what written in the documents.

In [505]:
extracted_value = [] # run this code to get the extracted results.
for i in metrics:
    extracted_value.append(get_metric(tables, i))
tuples = list(zip(metrics, extracted_value))
output = pd.DataFrame(tuples, columns=['Metrics','Extracted Value'])

output

,Metrics,Extracted Value
0,Application,47443.0
1,Enroll,1675.0
2,Accepted,N/A
3,SAT,N/A
4,Tuition,55473.0
5,Total operating revenues,9797.0
6,Total operating expenses,9307.0
7,Net Assets,490.0
8,Net income,N/A
9,Net operating surplus,N/A
